<img src="https://learn.deeplearning.ai/assets/dlai-logo.png"></img>

# Agentic Workflows

## Pattern For Highly Autonomous Agents - Planning Workflows

In this lab, you will build an agentic system that generates a short research report through planning, external tool usage, and feedback integration. Your workflow will involve:

### 👥 Agents

* **Planning Agent / Writer**: Creates an outline and coordinates tasks.
* **Research Agent**: Gathers external information using tools like Arxiv, Tavily, and Wikipedia.
* **Editor Agent**: Reflects on the report and provides suggestions for improvement.

### 🧰 Available Tools

You have access to:

* `arxiv_search_tool()`
* `tavily_search_tool()`
* `wikipedia_search_tool()`

Each tool can be called as part of an agent workflow. They are already implemented and available to call.

---

## ✅ Objective

Implement a function `generate_research_report(prompt: str) -> dict` that orchestrates the full workflow:

1. The **planner agent** creates a research plan.
2. The **research agent** fetches external content based on the plan.
3. The **planner** writes a first draft.
4. The **editor agent** reflects on the draft and provides feedback.
5. The **planner** revises the draft using the feedback.

You should use **tool calls** and **reasoning steps** where appropriate. Don’t hard-code any queries, the agents should generate them dynamically.

---

## 🧪 Evaluation

You’ll be graded based on:

* Whether each agent's function executes correctly.
* Whether the response type from each step is a valid string or dictionary.

---

### ✅ Starter Functions

You'll use this tool mapping and definitions (already provided in `research_tools.py`): 

### 🧰 Research Tools

By importing `research_tools`, you gain access to several search utilities:

- `research_tools.arxiv_search_tool(query)` → search academic papers from **arXiv**  
  *Example:* `research_tools.arxiv_search_tool("neural networks for climate modeling")`

- `research_tools.tavily_search_tool(query)` → perform web searches with the **Tavily API**  
  *Example:* `research_tools.tavily_search_tool("latest trends in sunglasses fashion")`

- `research_tools.wikipedia_search_tool(query)` → retrieve summaries from **Wikipedia**  
  *Example:* `research_tools.wikipedia_search_tool("Ensemble Kalman Filter")`

Run the cell below to make them available.


In [ ]:
# =========================
# Imports
# =========================

# --- Standard library 
from datetime import datetime
import re
import json


# --- Third-party ---
from IPython.display import Markdown, display
from aisuite import Client

# --- Local / project ---
import research_tools

### 🤖 Initialize client

Create a shared client instance for upcoming calls.

`client = Client()`

In [ ]:
client = Client()

### 🧠 Exercise 1: Implement the Planner Agent

Create a function called `planner_agent(topic: str) -> List[str]` that generates a **step-by-step research plan** as a Python list of strings.

Each step must:

* Be executable by one of the available agents (`research_agent`, `writer_agent`, `editor_agent`).
* Be clearly written and atomic (not a compound task).
* Avoid unrelated tasks like file handling or installing packages.
* End with a final step that **generates a Markdown document** with the research report.

✅ Use the following model: `"openai:o4-mini"`
✅ Use a temperature of `1.0` to allow creative planning.

In [15]:
def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """
    prompt = f"""
You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

🧠 Available agents:
- A research agent who can search the web, Wikipedia, and arXiv.
- A writer agent who can draft research summaries.
- An editor agent who can reflect and revise the drafts.

🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
✅ DO assume tool use is available.
✅ DO NOT include explanation text — return ONLY the Python list.
✅ The final step should be to generate a Markdown document containing the complete research report.

Topic: "{topic}"
"""

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
    )

    # ⚠️ Evaluate only if the environment is safe
    steps = eval(response.choices[0].message.content.strip())
    return steps


In [16]:
steps = planner_agent("The ensemble Kalman filter for time series forecasting")

In [ ]:
steps

### 🔍 Exercise 2: Implement the Research Agent

Complete the function by replacing all `None` values with the correct arguments:

* Use the provided `model` and `messages`.
* Pass the tool definitions (`arxiv_search_tool`, `tavily_search_tool`, `wikipedia_search_tool`).
* Enable tool calls automatically with `tool_choice="auto"`.
* Limit to **12 iterations** (`max_turns=12`).
* Return the assistant’s final message content.


In [ ]:
def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """
    Ejecuta una tarea de investigación usando herramientas con aisuite (sin bucle manual).
    """
    print("==================================")
    print("🔍 Research Agent")
    print("==================================")

    prompt = f"""
You are a research assistant with access to the following tools:
- arxiv_tool: for finding academic papers
- tavily_tool: for general web search
- wikipedia_tool: for encyclopedic knowledge

Task:
{task}

Today is {datetime.now().strftime('%Y-%m-%d')}.
"""

    messages = [{"role": "user", "content": prompt.strip()}]
    tools = [research_tools.arxiv_search_tool, research_tools.tavily_search_tool, research_tools.wikipedia_search_tool]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_turns=12  # 🔁 The model can use tools multiple times
        )
        content = response.choices[0].message.content
        print("✅ Output:\n", content)
        return (content, messages) if return_messages else content

    except Exception as e:
        print("❌ Error:", e)
        return f"[Model Error: {str(e)}]"


### ✍️ Exercise 3: Implement the Writer Agent

Create a function `writer_agent(task: str) -> str` that handles writing tasks like drafting sections or summarizing content.

Your implementation must:

* Use the **`client.chat.completions.create()`** interface.
* Include a system prompt:
  `"You are a writing agent specialized in generating well-structured academic or technical content."`
* Use `temperature=1.0` for creativity.
* Set the **default model to `o4-mini`**.
* Return the final content from the assistant message.

In [ ]:
# GRADED FUNCTION: writer_agent
### START CODE HERE ###
def writer_agent(task: str, model: str = "openai:gpt-4o") -> str: # @REPLACE def writer_agent(task: str, model: str = None) -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")

    
    messages = [ # @KEEP messages = [
        {
            "role": "system", # @KEEP "role": "system",
            "content": "You are a writing agent specialized in generating well-structured academic or technical content." # @REPLACE "content": None
        },
        {
            "role": "user", # @KEEP "role": "user",
            "content": task # @REPLACE "content": None
        }
    ]

    response = client.chat.completions.create( # @KEEP response = client.chat.completions.create(
        model=model, # @REPLACE model=None,
        messages=messages, # @REPLACE messages=None,
        temperature=1.0 # @REPLACE temperature=None
    )

    return response.choices[0].message.content
### END CODE HERE ###


### 🧠 Exercise 4: Implement the Editor Agent

Complete the function `editor_agent(task: str) -> str` by replacing all `None` values with the correct arguments.

Your implementation must:

* Call **`client.chat.completions.create()`**.
* Define `messages` with:

  * A **system prompt**:
    `"You are an editor agent. Your job is to reflect on, critique, or improve existing drafts."`
  * A **user prompt** containing the `task`.
* Use `temperature=0.7`.
* Set the **default model to `o4-mini`**.
* Return the assistant’s final message content.


In [ ]:
# GRADED FUNCTION: editor_agent
### START CODE HERE ###
def editor_agent(task: str, model: str = "openai:gpt-4o") -> str:  # @REPLACE def editor_agent(task: str, model: str = None) -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")
    
    messages = [  # @KEEP messages = [
        {
            "role": "system",  # @KEEP "role": "system",
            "content": "You are an editor agent. Your job is to reflect on, critique, or improve existing drafts."  # @REPLACE "content": None
        },
        {
            "role": "user",  # @KEEP "role": "user",
            "content": task  # @REPLACE "content": None
        }
    ]

    response = client.chat.completions.create(  # @KEEP response = client.chat.completions.create(
        model=model,  # @REPLACE model=None,
        messages=messages,  # @REPLACE messages=None,
        temperature=0.7  # @REPLACE temperature=None
    )

    return response.choices[0].message.content
### END CODE HERE ###


### 🎯 Exercise 5: The Executor Agent

The `executor_agent` manages the workflow by executing each step of a given plan. It:

1. Decides **which agent** (`research_agent`, `writer_agent`, or `editor_agent`) should handle the step.
2. Builds context from the outputs of previous steps.
3. Sends the enriched task to the selected agent.
4. Collects and stores the results in a shared history.

👉 **Do not implement or modify this function.** It is already provided as the orchestration component of the multi-agent pipeline.

In [21]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
    # puedes agregar más si lo deseas
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    # Quitar bloque tipo ```json ... ```
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()


In [22]:
def executor_agent(plan_steps: list[str], model: str = "openai:gpt-4o"):
    history = []

    print("==================================")
    print("🎯 Editor Agent")
    print("==================================")

    for i, step in enumerate(plan_steps):
        # Paso 1: Determinar el agente y la tarea
        agent_decision_prompt = f"""
You are an execution manager for a multi-agent research team.

Given the following instruction, identify which agent should perform it and extract the clean task.

Return only a valid JSON object with two keys:
- "agent": one of ["research_agent", "editor_agent", "writer_agent"]
- "task": a string with the instruction that the agent should follow

Only respond with a valid JSON object. Do not include explanations or markdown formatting.

Instruction: "{step}"
"""
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}],
            temperature=0,
        )

        # 🧼 Limpieza del bloque JSON
        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)

        agent_name = agent_info["agent"]
        task = agent_info["task"]

        # Paso 2: Construir el contexto con outputs anteriores
        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}" 
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""You are {agent_name}.

Here is the context of what has been done so far:
{context}

Your next task is:
{task}
"""

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        # Paso 3: Ejecutar el agente correspondiente
        if agent_name in agent_registry:
            output = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
        else:
            output = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))

        print(f"✅ Output:\n{output}")

    return history



In [ ]:
executor_history = executor_agent(steps)

In [ ]:
md = executor_history[-1][-1].strip("`")  
display(Markdown(md))